# Data Analysis - Project

---

In this notebook we cover all the previous classes - go through what we learned Pandas functionality and apply them to real world data to analyze the USA presidents dataset and draw helpful insights based on analysis. After the data analysis process we discuss about the way how the results and what results must be presented and the possible improvements. At the very last part, we cover Pandas GUI - Graphical User Interface.



$$
$$


### Lecture outline

---


* Fully fledged data analysis


* Presenting results and insights


* Discussion about improvements


* Graphical User Interface for Pandas


# > The code is not optimized in any direction!!!

### DRY - Don't Repeat Yourself
### KISS - Keep It Stupid Simple

In [1]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

import plotly.express as px

# Data Processing


---

Usually, in this stage we clean and process data in order to have it in an appropriate form. Most of the time, data cleaning and processing takes 80% of data scientist's time and is the most tedious process. However, this is the step what we makes TRUE data scientists. Because, you can copy-paste code to build the Machine Learning models bu you cannot copy-paste code for data cleaning. This is where the true art starts.

## Read Data

In [2]:
df = pd.read_csv("data/presidents.csv")

In [3]:
df.shape # We have 44 rows and 8 columns

(44, 8)

In [4]:
df.dtypes # All columns are represented as string

#                                  int64
President                         object
Born                              object
  Age atstart of presidency       object
Age atend of presidency           object
Post-presidencytimespan           object
      Died                        object
Age                               object
dtype: object

In [5]:
df.head()

,#,President,Born,Age atstart of presidency,Age atend of presidency,Post-presidencytimespan,Died,Age
0,1,George Washington,"Feb 22, 1732[a]","57 years, 67 daysApr 30, 1789","65 years, 10 daysMar 4, 1797","2 years, 285 days","Dec 14, 1799","67 years, 295 days"
1,2,John Adams,"Oct 30, 1735[a]","61 years, 125 daysMar 4, 1797","65 years, 125 daysMar 4, 1801","25 years, 122 days","Jul 4, 1826","90 years, 247 days"
2,3,Thomas Jefferson,"Apr 13, 1743[a]","57 years, 325 daysMar 4, 1801","65 years, 325 daysMar 4, 1809","17 years, 122 days","Jul 4, 1826","83 years, 82 days"
3,4,James Madison,"Mar 16, 1751[a]","57 years, 353 daysMar 4, 1809","65 years, 353 daysMar 4, 1817","19 years, 116 days","Jun 28, 1836","85 years, 104 days"
4,5,James Monroe,"Apr 28, 1758","58 years, 310 daysMar 4, 1817","66 years, 310 daysMar 4, 1825","6 years, 122 days","Jul 4, 1831","73 years, 67 days"


## Process columns

---

Let deal with DataFrame columns. Rename them and remove leading and trailing spaces if any.

In [6]:
df.columns # Columns contain leading and trailing spaces

Index(['#', 'President            ', 'Born', '  Age atstart of presidency   ',
       'Age atend of presidency', 'Post-presidencytimespan', '      Died',
       'Age'],
      dtype='object')

In [7]:
df.columns = df.columns.str.strip() # Remove spaces - same as TRIM function in Excel


df.columns

Index(['#', 'President', 'Born', 'Age atstart of presidency',
       'Age atend of presidency', 'Post-presidencytimespan', 'Died', 'Age'],
      dtype='object')

In [8]:
column_mapping = {"President": "president",
                  "Born": "birth_date",
                  "Age atstart of presidency": "age_at_start",
                  "Age atend of presidency": "age_at_end",
                  "Post-presidencytimespan": "post_presidency_timespan",
                  "Died": "death_date", "Age": "age_at_death"}



df = df.rename(column_mapping, axis=1) # Rename columns

### Feature Description

---

* `president` - First and last name of the president


* `birth_date` - Date of birth


* `age_at_start` - The age at the start of presidency


* `age_at_end` - The age at the end of presidency


* `post_presidency_timespan` - Th period between death and presidency end


* `death_date` - Death date


* `age_at_death` - The age at the moment of death

## Drop redundant column

---

The column `#` replicates the index of DataFrame and hance is redundant and we have to remove it.

In [9]:
df = df.drop("#", axis=1)

## Remove footnotes

---

Some columns contain footnote such as `[a]` in `birth_date` column or `[e]` in `age_at_end` column. We have to remove them as they do not carry any information and even might cause some issues.

In [10]:
birth_date = (df["birth_date"].str.split("[", expand=True)
                              .drop(1, axis=1)
                              .rename({0: "birth_date"}, axis=1))


age_at_end = (df["age_at_end"].str.split("[", expand=True)
                              .drop(1, axis=1)
                              .rename({0: "age_at_end"}, axis=1))


post_presidency_timespan = (df["post_presidency_timespan"].str.split("[", expand=True)
                                                          .drop(1, axis=1)
                                                          .rename({0: "post_presidency_timespan"}, axis=1))

We removed the footnotes but did not change the columns in the initial DataFrame. Note also, that we save processed columns as separate DataFrame. So we need to drop these columns from initial DataFrame and add processed ones instead.

In [11]:
df = df.drop(["birth_date", "age_at_end", "post_presidency_timespan"], axis=1) # Drop columns

In [12]:
df = pd.concat([df, birth_date, age_at_end, post_presidency_timespan], axis=1) # Concatenate processed column

## Split Columns


---

The values of columns `age_at_start` and `age_at_end` consists of two parts: the first part is the age of the president and the second part is the date the president hold the office - White House and left the office, respectively. It's better to split these two columns into two parts, actual age and the date.


To split these columns we have to figure out the common symbol or character on which we perform the split operation. If we observe, such a common character is `days` inside each value for each of those columns. Under common I mean the character or symbol which does not change across rows.

In [13]:
df[["age_at_start", "age_at_end"]].head()

,age_at_start,age_at_end
0,"57 years, 67 daysApr 30, 1789","65 years, 10 daysMar 4, 1797"
1,"61 years, 125 daysMar 4, 1797","65 years, 125 daysMar 4, 1801"
2,"57 years, 325 daysMar 4, 1801","65 years, 325 daysMar 4, 1809"
3,"57 years, 353 daysMar 4, 1809","65 years, 353 daysMar 4, 1817"
4,"58 years, 310 daysMar 4, 1817","66 years, 310 daysMar 4, 1825"


In [14]:
age_start = (df["age_at_start"].str.split("days", expand=True)
                               .rename({0: "age_at_start", 1: "presidency_start_date"}, axis=1))


age_end = (df["age_at_end"].str.split("days", expand=True)
                           .rename({0: "age_at_end", 1: "presidency_end_date"}, axis=1))

Now, drop `age_at_start` and `age_at_end` columns and insert new derived columns instead.

In [15]:
df = df.drop(["age_at_start", "age_at_end"], axis=1) # Drop columns

In [16]:
df = pd.concat([df, age_start, age_end], axis=1) # Add new columns

In [17]:
df.head()

,president,death_date,age_at_death,birth_date,post_presidency_timespan,age_at_start,presidency_start_date,age_at_end,presidency_end_date
0,George Washington,"Dec 14, 1799","67 years, 295 days","Feb 22, 1732","2 years, 285 days","57 years, 67","Apr 30, 1789","65 years, 10","Mar 4, 1797"
1,John Adams,"Jul 4, 1826","90 years, 247 days","Oct 30, 1735","25 years, 122 days","61 years, 125","Mar 4, 1797","65 years, 125","Mar 4, 1801"
2,Thomas Jefferson,"Jul 4, 1826","83 years, 82 days","Apr 13, 1743","17 years, 122 days","57 years, 325","Mar 4, 1801","65 years, 325","Mar 4, 1809"
3,James Madison,"Jun 28, 1836","85 years, 104 days","Mar 16, 1751","19 years, 116 days","57 years, 353","Mar 4, 1809","65 years, 353","Mar 4, 1817"
4,James Monroe,"Jul 4, 1831","73 years, 67 days","Apr 28, 1758","6 years, 122 days","58 years, 310","Mar 4, 1817","66 years, 310","Mar 4, 1825"


$$
$$

Some columns contain `days` component along with year. It's better to split these columns and will have year and days as a separate parts. That will make analysis process more smooth. Such columns are: `age_at_death`, `post_presidency_timespan`, `age_at_start`, `age_at_end`

$$
$$

In [18]:
df[["age_at_death", "post_presidency_timespan", "age_at_start", "age_at_end"]].head()

,age_at_death,post_presidency_timespan,age_at_start,age_at_end
0,"67 years, 295 days","2 years, 285 days","57 years, 67","65 years, 10"
1,"90 years, 247 days","25 years, 122 days","61 years, 125","65 years, 125"
2,"83 years, 82 days","17 years, 122 days","57 years, 325","65 years, 325"
3,"85 years, 104 days","19 years, 116 days","57 years, 353","65 years, 353"
4,"73 years, 67 days","6 years, 122 days","58 years, 310","66 years, 310"


In [19]:
age_at_death = (df["age_at_death"].str.rstrip("days")
                                  .str.split("years,", expand=True)
                                  .rename({0: "age_at_death_year",
                                           1: "age_at_death_days"},
                                          axis=1))

`post_presidency_timespan` column contains some uncommon values such as `1 year, 259 days` and `103 days`. So we could not use the same approach we used above. To deal such a situation we have to use `Regular Expression`.

In [20]:
post_presidency_timespan = (df["post_presidency_timespan"].str.rstrip("days")
                                                          .str.replace("year[s]?", "", regex=True)
                                                          .str.split(",", expand=True)
                                                          .rename({0: "post_presidency_timespan_year",
                                                                   1: "post_presidency_timespan_days"},
                                                                  axis=1))


post_presidency_timespan.loc[10] = [np.nan, 103] # Swap the values for one row

In [21]:
age_at_start = (df["age_at_start"].str.split("years,", expand=True)
                                  .rename({0: "age_at_start_year",
                                           1: "age_at_start_days"},
                                          axis=1))

In [22]:
age_at_end = (df["age_at_end"].str.split("years,", expand=True)
                                  .rename({0: "age_at_end_year",
                                           1: "age_at_end_days"},
                                          axis=1))

**drop old columns and add new ones**

In [23]:
df = df.drop(["age_at_death", "post_presidency_timespan", "age_at_start", "age_at_end"], axis=1) # Drop columns

In [24]:
df = pd.concat([df, age_at_death, post_presidency_timespan, age_at_start, age_at_end], axis=1) # Add new columns

In [25]:
df.head()

,president,death_date,birth_date,presidency_start_date,presidency_end_date,age_at_death_year,age_at_death_days,post_presidency_timespan_year,post_presidency_timespan_days,age_at_start_year,age_at_start_days,age_at_end_year,age_at_end_days
0,George Washington,"Dec 14, 1799","Feb 22, 1732","Apr 30, 1789","Mar 4, 1797",67,295,2,285,57,67,65,10
1,John Adams,"Jul 4, 1826","Oct 30, 1735","Mar 4, 1797","Mar 4, 1801",90,247,25,122,61,125,65,125
2,Thomas Jefferson,"Jul 4, 1826","Apr 13, 1743","Mar 4, 1801","Mar 4, 1809",83,82,17,122,57,325,65,325
3,James Madison,"Jun 28, 1836","Mar 16, 1751","Mar 4, 1809","Mar 4, 1817",85,104,19,116,57,353,65,353
4,James Monroe,"Jul 4, 1831","Apr 28, 1758","Mar 4, 1817","Mar 4, 1825",73,67,6,122,58,310,66,310


## Type casting

---

The columns are represented as sting objects. We have to convert them into appropriate type.

In [26]:
df.dtypes

president                        object
death_date                       object
birth_date                       object
presidency_start_date            object
presidency_end_date              object
age_at_death_year                object
age_at_death_days                object
post_presidency_timespan_year    object
post_presidency_timespan_days    object
age_at_start_year                object
age_at_start_days                object
age_at_end_year                  object
age_at_end_days                  object
dtype: object

### DateTime objects

---

Pandas supports `datetime object` - meaning that we can convert string representation of date into appropriate type and then operate on this object by using different methods.


The candidates for this conversion are: `death_date`, `birth_date`, `presidency_start_date`, and `presidency_end_date`

In [27]:
df["death_date"] = pd.to_datetime(df["death_date"].str.strip().str.replace("(living)", "", regex=False))

In [28]:
df["birth_date"] = pd.to_datetime(df["birth_date"].str.strip())

In [29]:
df["presidency_start_date"] = pd.to_datetime(df["presidency_start_date"].str.strip())

In [30]:
df["presidency_end_date"] = pd.to_datetime(df["presidency_end_date"].str.strip())

### Numeric objects

---

We have columns which are clearly numeric. However, they are interpreted as strings by Pandas due to a fact that Pandas cannot type cast automatically.

The candidates for numeric type are all columns except datetime columns and `president` column.

In [31]:
numeric_cols = ["age_at_death_year", "age_at_death_days",
                "post_presidency_timespan_year", "post_presidency_timespan_days",
               "age_at_start_year", "age_at_start_days",
               "age_at_end_year", "age_at_end_days"]

In [34]:
df[numeric_cols] = df[numeric_cols].apply(lambda x: x.str.strip()) # Apply strip function to all columns

In [37]:
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric) # Apply type casting

In [42]:
df.dtypes

president                                object
death_date                       datetime64[ns]
birth_date                       datetime64[ns]
presidency_start_date            datetime64[ns]
presidency_end_date              datetime64[ns]
age_at_death_year                         int64
age_at_death_days                         int64
post_presidency_timespan_year           float64
post_presidency_timespan_days           float64
age_at_start_year                         int64
age_at_start_days                         int64
age_at_end_year                           int64
age_at_end_days                           int64
dtype: object

## Reorder Columns

---

Let reorder columns to have them in logical order

In [45]:
df.head()

,president,death_date,birth_date,presidency_start_date,presidency_end_date,age_at_death_year,age_at_death_days,post_presidency_timespan_year,post_presidency_timespan_days,age_at_start_year,age_at_start_days,age_at_end_year,age_at_end_days
0,George Washington,1799-12-14,1732-02-22,1789-04-30,1797-03-04,67,295,2.0,285.0,57,67,65,10
1,John Adams,1826-07-04,1735-10-30,1797-03-04,1801-03-04,90,247,25.0,122.0,61,125,65,125
2,Thomas Jefferson,1826-07-04,1743-04-13,1801-03-04,1809-03-04,83,82,17.0,122.0,57,325,65,325
3,James Madison,1836-06-28,1751-03-16,1809-03-04,1817-03-04,85,104,19.0,116.0,57,353,65,353
4,James Monroe,1831-07-04,1758-04-28,1817-03-04,1825-03-04,73,67,6.0,122.0,58,310,66,310


In [47]:
columnsTitles = ["president", "presidency_start_date", "presidency_end_date",
                "birth_date", "age_at_start_year", "age_at_start_days",
                "age_at_end_year", "age_at_end_days",
                "post_presidency_timespan_year", "post_presidency_timespan_days",
                "death_date", "age_at_death_year", "age_at_death_days"]

In [49]:
df = df.reindex(columns=columnsTitles)

In [50]:
df.head()

,president,presidency_start_date,presidency_end_date,birth_date,age_at_start_year,age_at_start_days,age_at_end_year,age_at_end_days,post_presidency_timespan_year,post_presidency_timespan_days,death_date,age_at_death_year,age_at_death_days
0,George Washington,1789-04-30,1797-03-04,1732-02-22,57,67,65,10,2.0,285.0,1799-12-14,67,295
1,John Adams,1797-03-04,1801-03-04,1735-10-30,61,125,65,125,25.0,122.0,1826-07-04,90,247
2,Thomas Jefferson,1801-03-04,1809-03-04,1743-04-13,57,325,65,325,17.0,122.0,1826-07-04,83,82
3,James Madison,1809-03-04,1817-03-04,1751-03-16,57,353,65,353,19.0,116.0,1836-06-28,85,104
4,James Monroe,1817-03-04,1825-03-04,1758-04-28,58,310,66,310,6.0,122.0,1831-07-04,73,67


## Add Party Affiliation and Birth Place


---

Pandas can read HTML tables from the website. Here, I use this functionality to enrich our data with the party affiliation and birth place of the USA presidents. However, these data is messy and it needs separate processing.

In [60]:
party = pd.read_html("https://www.britannica.com/topic/Presidents-of-the-United-States-1846696")[0]

In [162]:
birth_place = pd.read_html("https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States_by_home_state")[0]

### Process Political Party Affiliation

In [66]:
party.head()

,Unnamed: 0,no.,president,birthplace,political party,term
0,NaN,1,George Washington,Va.,Federalist,1789–97
1,NaN,2,John Adams,Mass.,Federalist,1797–1801
2,NaN,3,Thomas Jefferson,Va.,Democratic-Republican,1801–09
3,NaN,4,James Madison,Va.,Democratic-Republican,1809–17
4,NaN,5,James Monroe,Va.,Democratic-Republican,1817–25


Drop unnecessary columns

In [72]:
party = party.drop(["Unnamed: 0", "no.", "birthplace", "term"], axis=1)

Remove last three row as they contain extra redundant information.

In [78]:
party = party.drop([44, 45, 46], axis=0)

Remove leading and trailing spaces

In [81]:
party = party.apply(lambda x: x.str.strip())

#### Merge `party` DataFrame with our initial DataFrame

In [84]:
df = df.merge(party["political party"], left_index=True, right_index=True)

In [85]:
df.head()

,president,presidency_start_date,presidency_end_date,birth_date,age_at_start_year,age_at_start_days,age_at_end_year,age_at_end_days,post_presidency_timespan_year,post_presidency_timespan_days,death_date,age_at_death_year,age_at_death_days,political party
0,George Washington,1789-04-30,1797-03-04,1732-02-22,57,67,65,10,2.0,285.0,1799-12-14,67,295,Federalist
1,John Adams,1797-03-04,1801-03-04,1735-10-30,61,125,65,125,25.0,122.0,1826-07-04,90,247,Federalist
2,Thomas Jefferson,1801-03-04,1809-03-04,1743-04-13,57,325,65,325,17.0,122.0,1826-07-04,83,82,Democratic-Republican
3,James Madison,1809-03-04,1817-03-04,1751-03-16,57,353,65,353,19.0,116.0,1836-06-28,85,104,Democratic-Republican
4,James Monroe,1817-03-04,1825-03-04,1758-04-28,58,310,66,310,6.0,122.0,1831-07-04,73,67,Democratic-Republican


### Birth Place Data

In [163]:
birth_place.head()

,Date of birth,President,Birthplace,State† of birth,In office
0,"February 22, 1732",George Washington,Westmoreland County,Virginia†,"(1st) April 30, 1789 – March 4, 1797"
1,"October 30, 1735",John Adams,Braintree,Massachusetts†,"(2nd) March 4, 1797 – March 4, 1801"
2,"April 13, 1743*",Thomas Jefferson,Shadwell,Virginia†,"(3rd) March 4, 1801 – March 4, 1809"
3,"March 16, 1751",James Madison,Port Conway,Virginia†,"(4th) March 4, 1809 – March 4, 1817"
4,"April 28, 1758",James Monroe,Monroe Hall,Virginia†,"(5th) March 4, 1817 – March 4, 1825"


Remove last two rows

In [164]:
birth_place = birth_place.drop([45, 46], axis=0)

Rename columns

In [166]:
column_mapping = {"Date of birth": "birth_date", "President": "president",
                 "Birthplace": "city", "State† of birth": "state"}


birth_place = birth_place.rename(column_mapping, axis=1)

Remove `†` character from the `state` column

In [167]:
birth_place["state"] = birth_place["state"].str.strip("†").str.strip()

Remove leading and trailing spaces

In [168]:
birth_place = birth_place.apply(lambda x: x.str.strip())

The order of rows in `birth_place` DataFrame is not set according to presidency order. Hence, we need to find at least one common column between `birth_place` and our initial DataFrame. That column could be `birth_date` as it is represented in both DataFrame.

In [169]:
birth_place["birth_date"] = pd.to_datetime(birth_place["birth_date"].str.rstrip("*").str.strip())

#### Merge `birth_place` DataFrame with our initial DataFrame

---

To perform the merge, we need to create temporary columns in both DataFrame and them merge these DataFrame using these columns.

In [171]:
birth = birth_place[["city", "state"]].copy(deep=True) # Select only necessary columns

In [172]:
birth.insert(0, column="temp", value=birth_place["birth_date"].dt.year) # Temporary column for merge

In [173]:
df["temp"] = df["birth_date"].dt.year # Temporary column for merge

In [176]:
df = df.merge(birth, how="inner", left_on="temp", right_on="temp").drop("temp", axis=1) # Perform merge

In [177]:
df.head()

,president,presidency_start_date,presidency_end_date,birth_date,age_at_start_year,age_at_start_days,age_at_end_year,age_at_end_days,post_presidency_timespan_year,post_presidency_timespan_days,death_date,age_at_death_year,age_at_death_days,political party,city,state
0,George Washington,1789-04-30,1797-03-04,1732-02-22,57,67,65,10,2.0,285.0,1799-12-14,67,295,Federalist,Westmoreland County,Virginia
1,John Adams,1797-03-04,1801-03-04,1735-10-30,61,125,65,125,25.0,122.0,1826-07-04,90,247,Federalist,Braintree,Massachusetts
2,Thomas Jefferson,1801-03-04,1809-03-04,1743-04-13,57,325,65,325,17.0,122.0,1826-07-04,83,82,Democratic-Republican,Shadwell,Virginia
3,James Madison,1809-03-04,1817-03-04,1751-03-16,57,353,65,353,19.0,116.0,1836-06-28,85,104,Democratic-Republican,Port Conway,Virginia
4,James Monroe,1817-03-04,1825-03-04,1758-04-28,58,310,66,310,6.0,122.0,1831-07-04,73,67,Democratic-Republican,Monroe Hall,Virginia


# Data Analysis

---

fsdkfjsdkfns

In [178]:
df.head()

,president,presidency_start_date,presidency_end_date,birth_date,age_at_start_year,age_at_start_days,age_at_end_year,age_at_end_days,post_presidency_timespan_year,post_presidency_timespan_days,death_date,age_at_death_year,age_at_death_days,political party,city,state
0,George Washington,1789-04-30,1797-03-04,1732-02-22,57,67,65,10,2.0,285.0,1799-12-14,67,295,Federalist,Westmoreland County,Virginia
1,John Adams,1797-03-04,1801-03-04,1735-10-30,61,125,65,125,25.0,122.0,1826-07-04,90,247,Federalist,Braintree,Massachusetts
2,Thomas Jefferson,1801-03-04,1809-03-04,1743-04-13,57,325,65,325,17.0,122.0,1826-07-04,83,82,Democratic-Republican,Shadwell,Virginia
3,James Madison,1809-03-04,1817-03-04,1751-03-16,57,353,65,353,19.0,116.0,1836-06-28,85,104,Democratic-Republican,Port Conway,Virginia
4,James Monroe,1817-03-04,1825-03-04,1758-04-28,58,310,66,310,6.0,122.0,1831-07-04,73,67,Democratic-Republican,Monroe Hall,Virginia


In [179]:
df["state"].value_counts()

Ohio              9
Virginia          8
New York          6
Massachusetts     6
North Carolina    2
Texas             2
Vermont           2
Nebraska          2
South Carolina    2
New Jersey        2
Arkansas          2
Connecticut       2
Georgia           2
California        2
Hawaii            1
Pennsylvania      1
Kentucky          1
Missouri          1
Illinois          1
Iowa              1
New Hampshire     1
Name: state, dtype: int64

In [180]:
df["city"].value_counts()

Braintree              3
New York City          3
New Haven              2
Delaware               2
Plains                 2
Yorba Linda            2
Waxhaws Region         2
Point Pleasant         2
Omaha                  2
Charles City County    2
Hope                   2
Caldwell               2
Milton                 2
Tampico                1
West Branch            1
Monroe Hall            1
Brookline              1
Plymouth               1
Port Conway            1
Hyde Park              1
Cove Gap               1
Fairfield              1
Raleigh                1
Barboursville          1
Niles                  1
Honolulu               1
North Bend             1
Cincinnati             1
Pineville              1
Kinderhook             1
Stonewall              1
Staunton               1
Hillsborough           1
Blooming Grove         1
Sinking Spring         1
Summerhill             1
Denison                1
Shadwell               1
Lamar                  1
Moreland Hills         1


In [184]:
pd.DataFrame(df["political party"].value_counts())

,political party
Republican,25
Democratic,19
Whig,4
Democratic-Republican,3
National Republican,2
Federalist,2
Democratic (Union),1
